In [1]:
data_file = "../data/drugs_29-Mar-2021-19_18_52_ATC_N02.txt"

In [2]:
fo = open(data_file,'r', encoding="ISO-8859-1")
header = fo.readline()
header = header.strip().split("\t")

In [3]:
header

['Drug ID',
 'Drug Name',
 'CAS Number',
 'Chemical Name',
 'New Molecular Entity',
 'Drug Interactions',
 'Comparative Efficacy',
 'Highest Development Phase',
 'Indication',
 'Molecular Formula',
 'Mechanism Of Action',
 'Organisation',
 'Pharmacodynamic Characteristics',
 'Route Of Administration',
 'Route Of Elimination',
 'Pharmacokinetic Characteristics',
 'Adverse Events',
 'Therapeutic Class (EphMRA)',
 'Therapeutic Class (WHO)',
 'Synonyms/Brand Names',
 'Antimicrobial Activity',
 'Immunogenicity',
 'Last Significant Update Date',
 'Therapeutic Area',
 'Orphan Designation',
 'License Availability',
 'Brands',
 'On Fastrack',
 'Fastrack country',
 'On PRIME',
 'PRIME country',
 'Biomarkers-Inclusion',
 'Biomarkers-Exclusion',
 'Biomarkers-Outcome Measure',
 'Biomarkers-Safety',
 'Biomarkers-not specified',
 'URL']

In [4]:
ind = 8; moa = 10

In [5]:
therapeutic = [header.index(x) for x in header if 'Therapeutic' in x]
print(therapeutic)

[17, 18, 23]


In [6]:
indicators = []
mechanisms = []
thEs = []
thWs = []
thAs = []

for line in fo:
    line = line.strip().split('\t')
    rowid = line[0]
    try:
        indicator = line[ind]
        mechanism = line[moa]
        thE = line[17]
        thW = line[18]
        thA = line[23]
    except:
        pass
    indicators.append(indicator.split(","))
    mechanisms.append(mechanism.split(","))
    thEs.append(thE.split(","))
    thWs.append(thW.split(","))
    thAs.append(thA.split(","))



In [7]:
indicators_flat = [item.strip('\" ').replace('\'','') for sublist in indicators for item in sublist]
indicators_flat = set(indicators_flat)

In [8]:
mechanisms_flat = [item.strip('\" ').replace('\'','') for sublist in mechanisms for item in sublist]
mechanisms_flat = set(mechanisms_flat)

In [9]:
thE_flat = [item.strip('\" ').replace('\'','') for sublist in thEs for item in sublist]
thE_flat = set(thE_flat)
thW_flat = [item.strip('\" ').replace('\'','') for sublist in thWs for item in sublist]
thW_flat = set(thW_flat)
thA_flat = [item.strip('\" ').replace('\'','') for sublist in thAs for item in sublist]
thA_flat = set(thA_flat)

In [10]:
len(mechanisms_flat)

650

In [11]:
len(indicators_flat)

487

In [12]:
print(len(thE_flat))
print(len(thW_flat))
print(len(thA_flat))

274
425
51


In [13]:
import pandas as pd

In [14]:
df = pd.read_table('../data/drugs_29-Mar-2021-19_18_52_ATC_N02.txt', encoding="ISO-8859-1")

In [15]:
df.fillna('', inplace=True)

In [16]:
df['Therapeutic Class (EphMRA)']

0       N2B (Non-Narcotics and Anti-Pyretics), N2C (An...
1                                         N2 (Analgesics)
2                                         N2 (Analgesics)
3       N2 (Analgesics), N3A (Anti-Epileptics), N5A (A...
4                                         N2 (Analgesics)
                              ...                        
2125                                      N2 (Analgesics)
2126    M1A (Anti-Rheumatics, Non-Steroidal), N2 (Anal...
2127                                      N2A (Narcotics)
2128                                      N2A (Narcotics)
2129                                      N2A (Narcotics)
Name: Therapeutic Class (EphMRA), Length: 2130, dtype: object

In [17]:
cleaned_indication = df.set_index('Drug ID').Indication.str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Indication')

In [18]:
cleaned_indication.Indication = cleaned_indication.Indication.str.strip(" ")

In [19]:
pd.set_option("min_rows", 20)
cleaned_indication.Indication.value_counts().head(20)

Pain                         1152
Neuropathic pain              419
Migraine                      243
Inflammation                  150
Postoperative pain            134
Cancer pain                   119
Epilepsy                       74
Diabetic neuropathies          72
Cancer                         64
Major depressive disorder      59
Musculoskeletal pain           58
Anxiety disorders              54
Parkinson's disease            53
Acute pain                     52
Osteoarthritis                 51
Rheumatoid arthritis           50
Postherpetic neuralgia         48
Inflammatory pain              43
Alzheimer's disease            42
Neurological disorders         42
Name: Indication, dtype: int64

In [20]:
ind = pd.get_dummies(cleaned_indication, prefix='IND').groupby(level=0).sum()

In [21]:
cleaned_mechanism = df.set_index('Drug ID')['Mechanism Of Action'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Mechanism')
cleaned_mechanism.Mechanism = cleaned_mechanism.Mechanism.str.strip(" ")
mch = pd.get_dummies(cleaned_mechanism, prefix='MCH').groupby(level=0).sum()

In [22]:
cleaned_therape = df.set_index('Drug ID')['Therapeutic Class (EphMRA)'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapE')
cleaned_therape.TherapE = cleaned_therape.TherapE.str.strip(" ")
the = pd.get_dummies(cleaned_therape, prefix='THE').groupby(level=0).sum()

In [23]:
cleaned_therape.TherapE.value_counts()

N2 (Analgesics)                                                     1219
N2B (Non-Narcotics and Anti-Pyretics)                                385
N2A (Narcotics)                                                      334
M1 (Anti-Inflammatory and Anti-Rheumatic Products)                   201
N2C (Anti-Migraine Preparations)                                     200
N7X (All other CNS drugs)                                            132
N7 (Other CNS Drugs)                                                 131
L1 (Antineoplastics)                                                  81
N3A (Anti-Epileptics)                                                 80
N6A (Anti-Depressants and Mood Stabilisers)                           79
                                                                    ... 
Medicinal Dentifrices etc)                                             1
L3A (Immunostimulating Agents Excluding Interferons)                   1
C7 (Beta-Blocking Agents)                          

In [24]:
cleaned_therapw = df.set_index('Drug ID')['Therapeutic Class (WHO)'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapW')
cleaned_therapw.TherapW = cleaned_therapw.TherapW.str.strip(" ")
thw = pd.get_dummies(cleaned_therapw, prefix='THW').groupby(level=0).sum()

In [25]:
cleaned_therapw.TherapW.value_counts()

N02 (Analgesics)                                        1144
N02B (Other Analgesics and Antipyretics)                 372
M01 (Antiinflammatory and Antirheumatic Products)        160
N02A (Opioids)                                           150
N07X (Other Nervous System Drugs)                        130
N02C (Antimigraine Preparations)                         126
N07 (Other Nervous System Drugs)                          98
L01 (Antineoplastic Agents)                               83
N04 (Anti-Parkinson Drugs)                                62
N06A (Antidepressants)                                    56
                                                        ... 
N06D-X01 (Memantine)                                       1
N03A-X09 (Lamotrigine)                                     1
V04C-K (Tests for pancreatic function)                     1
M04A-X (Other antigout preparations)                       1
C01D (Vasodilators Used in Cardiac Diseases)               1
N03A-F02 (Oxcarbazepine)

In [26]:
cleaned_therapa = df.set_index('Drug ID')['Therapeutic Area'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapA')
cleaned_therapa.TherapA = cleaned_therapa.TherapA.str.strip(" ")
tha = pd.get_dummies(cleaned_therapa, prefix='THA').groupby(level=0).sum()

In [27]:
cleaned_therapa.TherapA.value_counts()

Pain                                                1822
Advances in the Treatment of Nausea and Migraine     274
Neurological Disorders                               211
Rheumatic Disease                                    177
Inflammation                                         157
Cancer                                               116
Affective Disorders                                  114
Drug Delivery Systems                                110
Anxiety Disorders                                     92
Diabetes                                              86
Epilepsy and Seizure Disorders                        84
Parkinson's Disease and Movement Disorders            79
Musculoskeletal Disorders                             70
Alzheimer's Disease and Cognition Disorders           68
Skin Disorders                                        65
Drug Withdrawal                                       63
Respiratory Tract Disorders                           58
Genitourinary Disorders        

In [28]:
df = df.set_index('Drug ID').join(ind).join(mch).join(the).join(thw).join(tha)

In [29]:
df.to_csv('../data/drugs.txt',sep="\t")

In [30]:
ind.shape

(2130, 481)

In [31]:
mch.shape

(2130, 649)

# Market-Basket Analysis

In [32]:
#!conda install MLxtend --channel conda-forge --yes

In [33]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [34]:
frequent_indicators = apriori(ind, min_support=0.01, use_colnames=True, max_len=4)

In [35]:
frequent_indicators.sort_values(by=['support'],ascending=False)

,support,itemsets
25,0.540845,(IND_Pain)
22,0.196714,(IND_Neuropathic pain)
17,0.114085,(IND_Migraine)
14,0.070423,(IND_Inflammation)
28,0.062911,(IND_Postoperative pain)
44,0.055869,"(IND_Inflammation, IND_Pain)"
8,0.055869,(IND_Cancer pain)
12,0.034742,(IND_Epilepsy)
48,0.034272,"(IND_Pain, IND_Neuropathic pain)"
11,0.033803,(IND_Diabetic neuropathies)


In [36]:
rules = association_rules(frequent_indicators, metric="lift", min_threshold=1)


In [37]:
rules[ (rules['lift'] >= 2) &
       (rules['confidence'] >= 0.1) ].sort_values(by=['confidence'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,"(IND_Cancer, IND_Pain)",(IND_Inflammation),0.019249,0.070423,0.010329,0.536585,7.619512,0.008973,2.005930
10,(IND_Diabetic neuropathies),(IND_Neuropathic pain),0.033803,0.196714,0.014085,0.416667,2.118138,0.007435,1.377062
2,(IND_Cancer),(IND_Inflammation),0.030047,0.070423,0.012207,0.406250,5.768750,0.010091,1.565604
27,(IND_Cancer),"(IND_Inflammation, IND_Pain)",0.030047,0.055869,0.010329,0.343750,6.152836,0.008650,1.438677
12,(IND_Epilepsy),(IND_Migraine),0.034742,0.114085,0.010329,0.297297,2.605939,0.006365,1.260726
8,(IND_Cancer pain),(IND_Postoperative pain),0.055869,0.062911,0.011268,0.201681,3.205820,0.007753,1.173828
26,"(IND_Inflammation, IND_Pain)",(IND_Cancer),0.055869,0.030047,0.010329,0.184874,6.152836,0.008650,1.189942
9,(IND_Postoperative pain),(IND_Cancer pain),0.062911,0.055869,0.011268,0.179104,3.205820,0.007753,1.150124
3,(IND_Inflammation),(IND_Cancer),0.070423,0.030047,0.012207,0.173333,5.768750,0.010091,1.173330
28,(IND_Inflammation),"(IND_Cancer, IND_Pain)",0.070423,0.019249,0.010329,0.146667,7.619512,0.008973,1.149318


In [38]:
frequent_mechanisms = apriori(mch, min_support=0.01, use_colnames=True, max_len=4)
frequent_mechanisms.sort_values(by=['support'],ascending=False)

,support,itemsets
20,0.123474,(MCH_Undefined mechanism)
12,0.101878,(MCH_Opioid mu receptor agonists)
13,0.061972,(MCH_Opioid receptor agonists)
6,0.046009,(MCH_Cyclooxygenase inhibitors)
11,0.036150,(MCH_Opioid kappa receptor agonists)
5,0.032864,(MCH_Cyclo-oxygenase 2 inhibitors)
18,0.030047,(MCH_Sodium channel antagonists)
10,0.027230,(MCH_Opioid delta receptor agonists)
16,0.027230,(MCH_Serotonin 1D receptor agonists)
17,0.026291,(MCH_Serotonin uptake inhibitors)


In [39]:
rules_mch = association_rules(frequent_mechanisms, metric="lift", min_threshold=1)
rules_mch[ (rules_mch['lift'] >= 2) &
       (rules_mch['confidence'] >= 0.1) ].sort_values(by=['confidence'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,"(MCH_Opioid kappa receptor agonists, MCH_Opioi...",(MCH_Opioid mu receptor agonists),0.016432,0.101878,0.016432,1.000000,9.815668,0.014758,inf
9,(MCH_Serotonin 1B receptor agonists),(MCH_Serotonin 1D receptor agonists),0.018310,0.027230,0.016901,0.923077,33.899204,0.016403,12.646009
11,"(MCH_Opioid mu receptor agonists, MCH_Opioid d...",(MCH_Opioid kappa receptor agonists),0.018310,0.036150,0.016432,0.897436,24.825175,0.015770,9.397535
12,"(MCH_Opioid kappa receptor agonists, MCH_Opioi...",(MCH_Opioid delta receptor agonists),0.018779,0.027230,0.016432,0.875000,32.133621,0.015921,7.782160
2,(MCH_Opioid delta receptor agonists),(MCH_Opioid mu receptor agonists),0.027230,0.101878,0.018310,0.672414,6.600191,0.015536,2.741636
8,(MCH_Serotonin 1D receptor agonists),(MCH_Serotonin 1B receptor agonists),0.027230,0.018310,0.016901,0.620690,33.899204,0.016403,2.588092
1,(MCH_Opioid delta receptor agonists),(MCH_Opioid kappa receptor agonists),0.027230,0.036150,0.016432,0.603448,16.692790,0.015448,2.430578
13,(MCH_Opioid delta receptor agonists),"(MCH_Opioid kappa receptor agonists, MCH_Opioi...",0.027230,0.018779,0.016432,0.603448,32.133621,0.015921,2.474383
4,(MCH_Opioid kappa receptor agonists),(MCH_Opioid mu receptor agonists),0.036150,0.101878,0.018779,0.519481,5.099048,0.015096,1.869065
0,(MCH_Opioid kappa receptor agonists),(MCH_Opioid delta receptor agonists),0.036150,0.027230,0.016432,0.454545,16.692790,0.015448,1.783412


In [40]:
rules_mch['lhs items'] = [len(x) for x in rules_mch['antecedents']]
rules_mch[rules_mch['lhs items']>1].sort_values('lift', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,lhs items
12,"(MCH_Opioid kappa receptor agonists, MCH_Opioi...",(MCH_Opioid delta receptor agonists),0.018779,0.027230,0.016432,0.875000,32.133621,0.015921,7.782160,2
11,"(MCH_Opioid mu receptor agonists, MCH_Opioid d...",(MCH_Opioid kappa receptor agonists),0.018310,0.036150,0.016432,0.897436,24.825175,0.015770,9.397535,2
10,"(MCH_Opioid kappa receptor agonists, MCH_Opioi...",(MCH_Opioid mu receptor agonists),0.016432,0.101878,0.016432,1.000000,9.815668,0.014758,inf,2


In [41]:
df['Drug Name']

Drug ID
800004287                 Ganaxolone - Marinus Pharmaceuticals
800051342                                               VX 128
800048603    Research programme: Non narcotic analgesics - ...
800026736    Research programme: vesicular monoamine transp...
800051306                                              STR 324
800039110    Research programme: purinoceptor P2X3 antagoni...
800045567                    Nicotinamide riboside - ChromaDex
800019286                                             AMG 0101
800048963    Research programme: cannabinoid-based therapeu...
800047450                                             KAND 567
                                   ...                        
800003298                                           Tonazocine
800003819                                           WIN 521142
800003192                                          Trefentanil
800003687                                            MEN 10207
800003754                                      

In [42]:
frequent_all = apriori(mch.join(ind).join(tha), min_support=0.01, use_colnames=True, max_len=6)
frequent_all.sort_values(by=['support'],ascending=False)

,support,itemsets
77,0.855399,(THA_Pain)
46,0.540845,(IND_Pain)
227,0.538967,"(IND_Pain, THA_Pain)"
43,0.196714,(IND_Neuropathic pain)
200,0.196714,"(THA_Pain, IND_Neuropathic pain)"
55,0.128638,(THA_Advances in the Treatment of Nausea and M...
20,0.123474,(MCH_Undefined mechanism)
38,0.114085,(IND_Migraine)
181,0.113146,(THA_Advances in the Treatment of Nausea and M...
133,0.109859,"(THA_Pain, MCH_Undefined mechanism)"


In [43]:
rules_all = association_rules(frequent_all, metric="lift", min_threshold=1)
rules_all[ (rules_all['lift'] >= 2) &
       (rules_all['confidence'] >= 0.4) ].sort_values(by=['lift'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1235,(THA_Advances in the Treatment of Nausea and M...,"(IND_Epilepsy, IND_Migraine)",0.012207,0.010329,0.010329,0.846154,81.923077,0.010203,6.432864
1238,"(IND_Epilepsy, IND_Migraine)",(THA_Advances in the Treatment of Nausea and M...,0.010329,0.012207,0.010329,1.000000,81.923077,0.010203,inf
1239,"(THA_Epilepsy and Seizure Disorders, IND_Migra...",(THA_Advances in the Treatment of Nausea and M...,0.011268,0.011268,0.010329,0.916667,81.354167,0.010202,11.864789
1234,(THA_Advances in the Treatment of Nausea and M...,"(THA_Epilepsy and Seizure Disorders, IND_Migra...",0.011268,0.011268,0.010329,0.916667,81.354167,0.010202,11.864789
1473,"(IND_Cancer, IND_Inflammation)","(THA_Cancer, IND_Pain, THA_Inflammation)",0.012207,0.011268,0.010329,0.846154,75.096154,0.010191,6.426761
1464,"(THA_Cancer, IND_Pain, THA_Inflammation)","(IND_Cancer, IND_Inflammation)",0.011268,0.012207,0.010329,0.916667,75.096154,0.010191,11.853521
1475,"(IND_Cancer, THA_Inflammation)","(THA_Cancer, IND_Inflammation, IND_Pain)",0.013146,0.010798,0.010329,0.785714,72.763975,0.010187,4.616275
1462,"(THA_Cancer, IND_Inflammation, IND_Pain)","(IND_Cancer, THA_Inflammation)",0.010798,0.013146,0.010329,0.956522,72.763975,0.010187,22.697653
1470,"(THA_Cancer, IND_Inflammation)","(IND_Cancer, IND_Pain, THA_Inflammation)",0.013615,0.010798,0.010329,0.758621,70.254873,0.010182,4.098122
1467,"(IND_Cancer, IND_Pain, THA_Inflammation)","(THA_Cancer, IND_Inflammation)",0.010798,0.013615,0.010329,0.956522,70.254873,0.010182,22.686854


In [44]:
df['Adverse Events'].value_counts()

                                                                                                                                                                                                                                                                         1962
Dizziness, Somnolence                                                                                                                                                                                                                                                       3
Application site reactions                                                                                                                                                                                                                                                  3
Gastrointestinal disorders                                                                                                                                                                    

In [45]:
df[df['Adverse Events'].str.contains("Aggression")]

,Drug Name,CAS Number,Chemical Name,New Molecular Entity,Drug Interactions,Comparative Efficacy,Highest Development Phase,Indication,Molecular Formula,Mechanism Of Action,...,THA_Psychotic Disorders,THA_Respiratory Tract Disorders,THA_Rheumatic Disease,THA_Skin Disorders,THA_Thromboses,THA_Transplant Rejection,THA_Vaccines,THA_Vascular Disorders,THA_Viral Infections,THA_Women's Health
Drug ID,,,,,,,,,,,,,,,,,,,,,
800015670,Perampanel - Eisai,380917-97-5,"Benzonitrile, 2-[1',6'-dihydro-6'-oxo-1'-pheny...",Yes,,,Marketed,"Partial epilepsies, Tonic-clonic epilepsy, Sei...",C23 H15 N3 O,AMPA receptor antagonists,...,0,0,0,0,0,0,0,0,0,0
800002881,Vigabatrin - sanofi-aventis,60643-86-9,4-amino-5-hexenoic acid,Yes,,,Marketed,"Epilepsy, Infantile spasms, Complex partial ep...",C6 H11 N O2,4-aminobutyrate transaminase inhibitors,...,0,0,0,0,0,0,0,0,0,0
800000568,Venlafaxine - Pfizer,93413-69-5,"Cyclohexanol, 1-[2-(dimethylamino)-1-(4-methox...",Yes,,"³ Fluoxetine, &gt; Imipramine, ³ Trazodone",Marketed,"Major depressive disorder, Social phobia, Pani...",C17 H27 N O2,"Adrenergic uptake inhibitors, Serotonin uptake...",...,0,0,0,0,0,0,0,0,0,0
